In [1]:
import pandas as pd
import os
import re
import random
import asyncio
from playwright.async_api import Locator
import os
import glob
import pandas as pd


In [2]:
from playwright.async_api import Playwright, async_playwright
playwright = await async_playwright().start()

In [3]:

async def login():
    browser = await playwright.chromium.launch(headless=False)
    context = await browser.new_context()
    page = await context.new_page()
    await page.goto("https://marketing.toridori.me/campaigns/85296/campaign_entries")
    await page.goto("https://marketing.toridori.me/signin")
    await page.get_by_label("email").fill("mike.ogawa@techmo.jp")
    await asyncio.sleep(random.randint(3,5))
    await page.get_by_test_id("password").fill("1234567")
    await asyncio.sleep(random.randint(3,5))
    await page.get_by_label("ログイン").click()
    await asyncio.sleep(random.randint(3,6))
    return page
    
page = await login()

In [4]:
async def access_to_campaign():
    await page.goto("https://marketing.toridori.me/campaigns/85296/campaign_entries")
    await page.get_by_role("menuitem", name="キャンペーン一覧").nth(1).click()
    await asyncio.sleep(random.randint(1,2))
    await page.locator("a.MuiCardActionArea-root").click()
    await asyncio.sleep(random.randint(1,2))
    await page.locator("a.MuiButtonGroup-groupedHorizontal").nth(2).click()
    await asyncio.sleep(random.randint(1,5))
    return page

page = await access_to_campaign()

In [5]:
SRC_PATH = f"process_influencer.csv"
processed_influencer_df = pd.read_csv(SRC_PATH)
previous_df = processed_influencer_df[processed_influencer_df["結果"] == "O"].reset_index(drop=True)
previous_done_names = previous_df["name"].to_list()

async def close_icon(drawer_el: Locator):
    await drawer_el.locator(".MuiBox-root svg").nth(0).click()

def is_target_name(name: str) -> bool:
    return name in previous_done_names

def has_same_description(name: str, description: str) -> bool:
    sub_df = previous_df[previous_df["name"] == name]
    return len(sub_df[sub_df["desc"] == description])

def has_same_img(name: str, img_src: str) -> bool:
    sub_df = previous_df[previous_df["name"] == name]
    return len(sub_df[sub_df["img_src"] == img_src])

async def click_tr(tr_el: Locator) -> Locator:
    await tr_el.click()
    await asyncio.sleep(random.randint(3,5))

async def get_drawer_el(page: Locator) -> Locator:
    drawer_el = page.locator(".MuiPaper-root.MuiPaper-elevation.MuiDrawer-paperAnchorDockedRight")
    return drawer_el

async def extract_name(tr_el: Locator) -> str:
    title_els = await tr_el.locator("p.MuiTypography-root.MuiTypography-body1").all()
    if not title_els:
        return ""
    return await title_els[0].inner_text()

async def extract_description(drawer_el: Locator) -> str:
    desc = await drawer_el.locator(".MuiBox-root > .MuiBox-root > .MuiBox-root > p.MuiTypography-root.MuiTypography-body2").nth(-1).inner_text()
    return desc

async def extract_img(drawer_el: Locator) -> str:
    return await drawer_el.locator(".MuiAvatar-circular img").get_attribute("src")

async def click_candidate_select(drawer_el: Locator, page: Locator) -> str:
    # select = await drawer_el.get_by_text("採用する").inner_text()
    # print(select)
    await drawer_el.get_by_text("採用する").click()
    await asyncio.sleep(random.randint(1,2))
    await page.locator("[role=dialog]").get_by_text("採用を確定").click()
    await asyncio.sleep(random.randint(1,2))

async def per_row_process(page: Locator, tr_el: Locator):
    name = await extract_name(tr_el)
    if not is_target_name(name):
        return False
    await click_tr(tr_el)
    await asyncio.sleep(random.randint(2,4))
    drawer_el = await get_drawer_el(page)
    try:
        desc = await extract_description(drawer_el)
        img = await extract_img(drawer_el)
        await asyncio.sleep(random.randint(1,2))
        if not (has_same_description(name, desc) and has_same_img(name, img)):
            return False
        await click_candidate_select(drawer_el, page)
    except Exception as e:
        print(e)
    finally:
        await close_icon(drawer_el)
    return True


async def loop_per_table(page: Locator, res_list: list[dict[str, str]]) -> tuple[Locator, bool, bool]:
    res_list = []
    should_continue = True
    for tr in await page.locator("tbody tr").all():
        res = await per_row_process(page, tr)
        # if res:
        #     return page, res_list, False

    return page, res_list, should_continue

async def next_table(page: Locator) -> tuple[Locator, bool]:
    el = page.locator("button.MuiPaginationItem-root").nth(-1)
    disabled = await el.is_disabled()
    if not disabled:
        await el.click()
        await asyncio.sleep(random.randint(2,4))
    return page, disabled



In [7]:
MAX_PAGES = 50

for i in range(1, MAX_PAGES):
    res_min = []
    page, res_min, should_continue = await loop_per_table(page, res_min)
    page, is_disabled = await next_table(page)
    await asyncio.sleep(random.randint(3, 7))
    if is_disabled or not should_continue:
        break


In [8]:
await page.close()
await playwright.stop()